# BigQuery Parameterization

Google BigQuery Standard SQL [supports parameterization](https://cloud.google.com/bigquery/querying-data#running_parameterized_queries). It is interesting to be able to use Python variables defined in the notebook as parameter values for SQL.

This notebook is an example how to use parameterized queries.

## Data Preview

In [ ]:
%%bq query -n logs
SELECT * FROM `cloud-datalab-samples.httplogs.logs_20140615`

In [ ]:
%bq sample -q logs --count 10

In [ ]:
%%bq query -n endpoint
SELECT endpoint FROM `cloud-datalab-samples.httplogs.logs_20140615` GROUP BY endpoint

In [ ]:
%bq execute -q endpoint

# Parameterization within SQL queries

Parameters are declared in SQL queries using a `@name` syntax within the SQL, and then defining `name`'s value when executing the query.

In [ ]:
%%bq query -n endpoint_stats
SELECT *
FROM `cloud-datalab-samples.httplogs.logs_20140615`
WHERE endpoint = @endpoint
LIMIT 10

In [ ]:
%%bq execute -q endpoint_stats
parameters:
- name: endpoint
  type: STRING
  value: Interact2

This defined a SQL query with a string parameter named `endpoint`, which can be filled when executing the query. Let's give it some value in a separate cell:

In [ ]:
endpoint_val = 'Interact3'

In order to reference the variable defined above, Datalab offers the `$var` syntax, which can be invoked in the magic command:

In [ ]:
%%bq execute -q endpoint_stats
parameters:
- name: endpoint
  type: STRING
  value: $endpoint_val

This can also be achieved using the Python API instead of the magic commands (`%%bq`). In fact, those magic commands are just wrappers to make common tasks easier. This is how we will create and execute a parameterized query using the API:

In [ ]:
import google.datalab.bigquery as bq
endpoint_stats2 = bq.Query(sql='''
SELECT *
FROM `cloud-datalab-samples.httplogs.logs_20140615`
WHERE endpoint = @endpoint
LIMIT 10
''')

endpoint_value = 'Interact3'

query_parameters = [
  {
    'name': 'endpoint',
    'parameterType': {'type': 'STRING'},
    'parameterValue': {'value': endpoint_value}
  }
]

job = endpoint_stats2.execute(query_params=query_parameters)

job.result()

# Looking Ahead

Parameterization enables one part of the SQL and Python integration: being able to use values in Python code in the notebook, and passing them in as part of the query when retrieving data from BigQuery.

The next notebook will cover the other part of the SQL and Python integration: retrieving query results into the notebook for use with Python code.